# Databricks notebook source

## 1. Libraries

In [ ]:
!pip install u8darts[all]==0.29.0
!pip install mlflow==2.11.3
!pip install holidays==0.45
dbutils.library.restartPython()

In [ ]:
# Darts
from darts import TimeSeries
from darts.models.forecasting.lgbm import LightGBMModel
from darts import TimeSeries
from darts import metrics
from darts.dataprocessing.transformers import Scaler

# Analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Utilities
import pandas as pd
import numpy as np
import datetime
import mlflow
import mlflow.sklearn
from numpy import savetxt
from datetime import timedelta
import holidays
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import acf, pacf
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import pickle

# Notebook configuration
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_theme(style="whitegrid", palette="pastel")
import warnings
warnings.filterwarnings('ignore')


## 2. Utility Functions

In [ ]:
# Function to calculate Easter date
def calculate_easter(year):
    "Returns the Easter date for a given year."
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    month = (h + l - 7 * m + 114) // 31
    day = ((h + l - 7 * m + 114) % 31) + 1
    return pd.Timestamp(year, month, day)


# Function to calculate Carnival date
def calculate_carnival(year):
    "Returns the Carnival date for a given year."
    easter = calculate_easter(year)
    return easter - timedelta(days=47)
    

# Function to determine custom events
def custom_events(date):
    month_day = date.strftime('%m-%d')
    year = date.year

    # Carnival
    carnival = calculate_carnival(year)
    # Carnival Eve
    carnival_eve = carnival - timedelta(days=1)
    # Mother's Day (second Sunday of May)
    mothers_day = pd.Timestamp(year, 5, 1) + pd.DateOffset(weekday=6, weeks=1)
    # Mother's Day Eve
    mothers_day_eve = mothers_day - timedelta(days=1)
    # Valentines Day (June 12th in Brazil)
    valentines_day = pd.Timestamp(year, 6, 12)
    # Black Friday (fourth Friday of November)
    black_friday = pd.Timestamp(year, 11, 1) + pd.DateOffset(weekday=4, weeks=3)
    
    if date == carnival:
        return 'Carnival'
    elif date == carnival_eve:
        return 'Carnival Eve'
    elif month_day == mothers_day.strftime('%m-%d'):
        return 'Mothers Day'
    elif month_day == mothers_day_eve.strftime('%m-%d'):
        return 'Mothers Day Eve'
    elif month_day == valentines_day.strftime('%m-%d'):
        return 'Valentines Day'
    elif month_day == black_friday.strftime('%m-%d'):
        return 'Black Friday'
    else:
        return None


# Function to include events in holiday column
def include_events(row):
    if pd.isna(row['event']):
        return custom_events(row.name)
    else:
        return row['event']
    

# Function to replace values
def replace_event(row):
    if row['event'] not in events_to_consider:
        return 'Normal Day'
    else:
        return row['event']

def replace_or_add(lst, index, new_value):
    if index < len(lst):
        # If index exists in list, replace item
        lst[index] = new_value
    else:
        # If index does not exist, append new value to end of list
        lst.append(new_value)


# Function to remove outliers from metrics
def remove_outliers_zscore(data_list, threshold=3):
    # Remove null values
    data_np = np.array([x for x in data_list if x is not np.nan])

    if len(data_np) == 0:
        return []

    # Calculate z-scores and filter outliers
    z_scores = np.abs((data_np - np.mean(data_np)) / np.std(data_np))
    filtered_data = data_np[z_scores < threshold]

    return filtered_data.tolist()

    
def prepare_input(df, 
                  target_col='NET_VALUE',
                  numeric_features=None, 
                  use_event_flag=False, 
                  events_to_consider=None,
                  scale_trgt=False,
                  scale_cov=False,
                  scale_method=None,
                  dummy_day_of_week=False):
    '''
    Function to prepare model input data.

    Parameters:
    - df (pandas.DataFrame): DataFrame containing the data used.
    - target_col (str): Name of the target series column.
    - numeric_features (list): List with numerical feature columns to be used.
    - use_event_flag (boolean): Whether to use the event flag.
    - events_to_consider (list): List with event names to consider if used as variables.
    - scale_trgt (boolean): Whether to scale the target variable.
    - scale_cov (boolean): Whether to scale the covariates.
    - scale_method (str): Name of the method to be used when scaling data (minmax, power). 

    Returns:
    - list with the following items:
        - series (TimeSeries (DataArray)): target series.
        - series_cov (TimeSeries (DataArray)): covariate series.
        - transformer_trgt (darts.dataprocessing.transformers.scaler.Scaler): target series scaler for scale reversal.
    '''
    return_list = []

    # Generates main series
    series = TimeSeries.from_dataframe(
                                        df=df,
                                        value_cols=target_col,
                                        freq='D')
    return_list.append(series)

    # Initialize series_cov
    series_cov = None

    # Creates covariate series for numerical variables
    if numeric_features:
        series_cov =  TimeSeries.from_dataframe(
                                                df=df,
                                                value_cols=numeric_features,
                                                freq='D')
        replace_or_add(return_list, 1, series_cov)

    # Creates series for event_flag variable
    if use_event_flag:
        series_event_flag = TimeSeries.from_dataframe(
                                                        df=df,
                                                        value_cols='event_flag',
                                                        freq='D')
        
        series_cov = series_cov.stack(series_event_flag) if series_cov else series_event_flag
        replace_or_add(return_list, 1, series_cov)

    # Creates series with dummy of events to consider
    if events_to_consider:
        df['event'] = df.apply(replace_event, axis=1)
        df_dummy = pd.get_dummies(df['event'], prefix='event') * 1
        series_dummy = TimeSeries.from_dataframe(
                                                    df=df_dummy,
                                                    value_cols=df_dummy.columns.tolist(),
                                                    freq='D')
        
        series_cov = series_cov.stack(series_dummy) if series_cov else series_dummy
        replace_or_add(return_list, 1, series_cov)

    # Creates series with day of week dummy
    if dummy_day_of_week:
        df_dummy_dayofweek = pd.get_dummies(df['day_of_week'], prefix='day_of_week') * 1
        series_dummy_dayofweek = TimeSeries.from_dataframe(
                                                            df=df_dummy_dayofweek,
                                                            value_cols=df_dummy_dayofweek.columns.tolist(),
                                                            freq='D')
        series_cov = series_cov.stack(series_dummy_dayofweek) if series_cov else series_dummy_dayofweek
        replace_or_add(return_list, 1, series_cov)

    # Initialize transformers
    transformer_trgt = None
    transformer_cov = None

    # Performs scaling if desired
    if scale_trgt:
        if scale_method == 'minmax':
            scaler_trgt = MinMaxScaler()
        else:
            scaler_trgt = PowerTransformer()
            
        transformer_trgt = Scaler(scaler_trgt)
        series = transformer_trgt.fit_transform(series)
        replace_or_add(return_list, 0, series)
        replace_or_add(return_list, 2, transformer_trgt)

    if scale_cov and series_cov:
        if scale_method == 'minmax':
            scaler_cov = MinMaxScaler()
        else:
            scaler_cov = PowerTransformer()
            
        transformer_cov = Scaler(scaler_cov)
        series_cov = transformer_cov.fit_transform(series_cov)
        replace_or_add(return_list, 1, series_cov)


    return return_list


# Function to create the model
def create_model(use_covariates, lags, output_chunk_length, random_seed, n_estimators, multi_models, lags_past_covariates, lags_future_covariates):
    if use_covariates:
        return LightGBMModel(
            lags=lags,
            output_chunk_length=output_chunk_length,
            lags_past_covariates=lags_past_covariates,
            lags_future_covariates=lags_future_covariates,
            random_state=random_seed,
            n_estimators=n_estimators,
            multi_models=multi_models,
            force_row_wise=False,
            force_col_wise=False)
    else:
        return LightGBMModel(
            lags=lags,
            output_chunk_length=output_chunk_length,
            random_state=random_seed,
            n_estimators=n_estimators,
            multi_models=multi_models,
            force_row_wise=False,
            force_col_wise=True)


# Transform negative values to zero
def replace_negatives_with_zero(x):
    return np.maximum(x, 0)


# Function to calculate WAPE
def calculate_wape(y, yhat):
    """
    Calculates Weighted Absolute Percentage Error (WAPE).
    
    Args:
        y (list or numpy array): Actual values.
        yhat (list or numpy array): Predicted values.
    
    Returns:
        float: WAPE value.
        
    Description:
    This function calculates the Weighted Absolute Percentage Error (WAPE).
    
    WAPE takes into account both absolute error and percentage error between actual and predicted values. 
    Furthermore, it performs error weighting based on actual values. In seasonal sales time series scenarios, 
    seasonal peak periods are assigned a higher weight, so errors in these periods have a more significant impact on this metric.
    
    Weighting is performed by assigning higher weights to larger actual values, since errors in high demand periods 
    can have a more relevant impact on planning decisions and financial results.
    
    In the code section where weight is defined as 'weight = actual', it is considered that the weight of each observation 
    is equal to the actual value of that observation itself. This means that when calculating WAPE, absolute percentage errors 
    are multiplied by the actual values themselves before being summed to calculate total weighted error.
    """
    
    # Initializes variables to store total WAPE and total weights
    total_wape = 0
    total_weight = 0
    
    try:
        # Loop over actual (y) and predicted (yhat) values
        for i in range(len(y)):
            actual = y[i]  # Actual value
            predicted = yhat[i]  # Predicted value
            
            # Calculates absolute error between actual and predicted value
            absolute_error = abs(actual - predicted)
            
            # Calculates absolute percentage error
            absolute_percentage_error = absolute_error / actual
            
            # Calculates weight as the actual value (used for weighting)
            weight = actual
            
            # Updates total WAPE summing weighted percentage error
            total_wape += absolute_percentage_error * weight
            
            # Updates total weights
            total_weight += weight
        
        # Calculates final WAPE as weighted average of percentage errors
        wape = total_wape / total_weight * 100
    except:
        # If an exception occurs (e.g., division by zero), sets WAPE as infinite
        wape = np.inf
        
    return wape


# Function to perform predictions and calculate metrics
def predict_and_evaluate(model, serie_tgrt, serie_tgrt_test, series_cov, scale_trgt, transformer_trgt, target_col):
    df_preds = pd.DataFrame(columns=['NET_VALUE_predicted_05', 'NET_VALUE_predicted', 'NET_VALUE_predicted_95', 'NET_VALUE_real'])
    metrics_lists = {
        'mape': [], 'smape': [], 'ope': [], 'r2': [], 'rmse': [], 'wape': []
    }

    unique_dates = serie_tgrt_test.time_index.strftime('%Y-%m-%d').unique()
    min_time = serie_tgrt.time_index.min()

    progress_bar_general = tqdm(unique_dates, desc='Processing')
    for day in progress_bar_general:
        start_time = pd.Timestamp(day)
        start_real = pd.Timestamp(day) - pd.Timedelta(days=1)
        end_time = pd.Timestamp(day)

        serie_tgrt_filt = serie_tgrt.slice(min_time, start_real)
        serie_tgrt_test_filt = serie_tgrt_test.slice(start_time, end_time)
        current_series = serie_tgrt_filt
        predictions_05, predictions_median, predictions_95 = [], [], []

        if model.uses_past_covariates or model.uses_future_covariates:

            # Predicts
            pred = model.predict(
                series=current_series,
                past_covariates=series_cov,
                future_covariates=series_cov[future_variables],
                n=1,
                predict_likelihood_parameters=True)
            predictions_05.append(pred[f'{target_col}_q0.05'].values().item())
            predictions_median.append(pred[f'{target_col}_q0.50'].values().item())
            predictions_95.append(pred[f'{target_col}_q0.95'].values().item())
            current_series = current_series.append(pred[f'{target_col}_q0.50'])
        else:
            pred = model.predict(series=serie_tgrt_filt, n=len(serie_tgrt_test_filt))
            predictions_median = pred.values().flatten()

        pred_05_series = TimeSeries.from_times_and_values(serie_tgrt_test_filt.time_index, np.array(predictions_05))
        pred_series = TimeSeries.from_times_and_values(serie_tgrt_test_filt.time_index, np.array(predictions_median))
        pred_95_series = TimeSeries.from_times_and_values(serie_tgrt_test_filt.time_index, np.array(predictions_95))

        if scale_trgt:
            pred_05_series = transformer_trgt.inverse_transform(pred_05_series)
            pred_series = transformer_trgt.inverse_transform(pred_series)
            pred_95_series = transformer_trgt.inverse_transform(pred_95_series)
            serie_tgrt_test_filt = transformer_trgt.inverse_transform(serie_tgrt_test_filt)

        pred_05_series = pred_05_series.map(replace_negatives_with_zero)
        pred_series = pred_series.map(replace_negatives_with_zero)
        pred_95_series = pred_95_series.map(replace_negatives_with_zero)

        if start_time.weekday() < 5:
            df_trgt = pred_series.pd_dataframe()
            if df_trgt.index.weekday.unique() < 5:
                mape = metrics.mape(serie_tgrt_test_filt + 1, pred_series + 1)
                smape = metrics.smape(serie_tgrt_test_filt + 1, pred_series + 1)
                ope = metrics.ope(serie_tgrt_test_filt + 1, pred_series + 1)
                r2 = metrics.r2_score(serie_tgrt_test_filt + 1, pred_series + 1)
                rmse = metrics.rmse(serie_tgrt_test_filt + 1, pred_series + 1)
                wape = calculate_wape(serie_tgrt_test_filt + 1, pred_series + 1)

                metrics_lists['mape'].append(mape)
                metrics_lists['smape'].append(smape)
                metrics_lists['ope'].append(ope)
                metrics_lists['r2'].append(r2)
                metrics_lists['rmse'].append(rmse)
                metrics_lists['wape'].append(wape.values().squeeze().item())

        df_preds = pd.concat([df_preds, 
                              pred_05_series.stack(pred_series).stack(pred_95_series).pd_dataframe().rename(
                                  columns={'0': 'NET_VALUE_predicted_05', '0_1': 'NET_VALUE_predicted', '0_1_1': 'NET_VALUE_predicted_95'})])

    return df_preds, metrics_lists


## 2. Data Import and Preparation

In [ ]:
# Sales Series
str_select_sales = '''
                        SELECT *
                        FROM analytics.refined_sales_orders_agg
                        '''

df_series_sales = spark.sql(str_select_sales).toPandas()

df_series_sales.info()

In [ ]:
# Setting SYSTEM_TIMESTAMP column as DataFrame index
df_series_sales.set_index('SYSTEM_TIMESTAMP', inplace=True)
df_series_sales.sort_index(inplace=True)

# Converts granularity to Daily
df_series_sales_resampled = df_series_sales.resample('D').sum()



########## Outlier Treatment   ##########
# Decomposing the series for residual outlier verification
result = seasonal_decompose(df_series_sales_resampled['NET_VALUE'], model='additive', period=8)
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Calculates residual z-scores
z_scores = stats.zscore(residual.dropna())

# Gets outlier indices
zscore_threshold = 3 # Since the series has seasonality and high amplitude, a lower threshold would cut natural peaks
outliers = np.abs(z_scores) > zscore_threshold
outliers_index = residual.dropna().index[outliers]

# Residual interpolation at outlier points
residual_adjusted = residual.copy()
residual_adjusted[outliers_index] = np.nan
residual_adjusted = residual_adjusted.interpolate()
residual_adjusted = residual_adjusted.fillna(method='bfill').fillna(method='ffill')

# Reconstruction of adjusted time series
adjusted_time_series = trend + seasonal + residual_adjusted
df_series_sales_resampled['NET_VALUE_clean'] = adjusted_time_series
df_series_sales_resampled['NET_VALUE_clean'] = df_series_sales_resampled['NET_VALUE_clean'].fillna(df_series_sales_resampled['NET_VALUE'])



# Creating calendar variables
df_series_sales_resampled['year'] = df_series_sales_resampled.index.year
df_series_sales_resampled['month'] = df_series_sales_resampled.index.month
df_series_sales_resampled['week_of_year'] = df_series_sales_resampled.index.isocalendar().week
df_series_sales_resampled['day_of_week'] = df_series_sales_resampled.index.day_of_week #  Monday=0 and Sunday=6
df_series_sales_resampled['day_of_month'] = df_series_sales_resampled.index.day


# Inspecting
df_series_sales_resampled.info()

In [ ]:
display(df_series_sales_resampled.reset_index())

In [ ]:
# The operation below is not necessary, as model optimization showed such variables do not help with current parameterization.
# The code will be kept here in case it is needed at some future time.
'''
# Including holiday
brazil_holidays = holidays.Brazil()
df_series_sales_resampled['event'] = df_series_sales_resampled.index.map(lambda x: brazil_holidays.get(x, None))

# Adding custom holidays
df_series_sales_resampled['event'] = df_series_sales_resampled.apply(include_events, axis=1)

# Creating holiday flag
df_series_sales_resampled['event_flag'] = df_series_sales_resampled['event'].apply(lambda x: 1 if x is not None else 0)

df_series_sales_resampled['event'].fillna('Normal Day', inplace=True)
'''

In [ ]:
# Holidays to consider if dummies are created
events_to_consider = None

# Numerical variables
numeric_features = ['day_of_week', 'day_of_month', 'month']
future_variables = ['day_of_week', 'day_of_month', 'month']

# Generate day of week dummy
dummy_day_of_week = False

# Use event binary flag or not
use_event_flag = False

# Scale target series or not
scale_trgt = True

# Scale covariate series or not
scale_cov = False

# Scaling method (minmax or power)
scale_method = 'minmax'

# Target series column to predict
target_col = 'NET_VALUE'

# Generating series
return_list = prepare_input(df=df_series_sales_resampled, 
                            target_col=target_col,
                            numeric_features=numeric_features, 
                            use_event_flag=use_event_flag, 
                            events_to_consider=events_to_consider,
                            scale_trgt=scale_trgt,
                            scale_cov=scale_cov,
                            scale_method=scale_method,
                            dummy_day_of_week=dummy_day_of_week)

# Extracting objects from return list
serie_tgrt = return_list[0]
if use_event_flag or events_to_consider or numeric_features:
    series_cov = return_list[1]
if scale_trgt:
    transformer_trgt = return_list[2]

# Saving scaler to disk
artifacts_root = '/Workspace/Repos/DataScience/FORECAST_PROJECT/src/artifacts/'
with open(f'{artifacts_root}transformer_trgt.pkl', 'wb') as f:
    pickle.dump(transformer_trgt, f)


## 4. Training, Evaluation and Registration

In [ ]:
# Setting experiment
name = 'forecast_sales_daily'
model_name = 'LightGBM'
registered_model_name = 'LightGBM_forecast_sales_daily'
experiment_name = f"/Users/data.scientist@company.com/{name}_training"
mlflow.set_experiment(experiment_name)

In [ ]:
# Separating train and test
test_pct = 0.2 # Percentage of test sample
n_test = int(len(serie_tgrt) * test_pct)
dt_test_start = (serie_tgrt.time_index.max() - timedelta(days=n_test)).replace(hour=0, minute=0, second=0)
serie_tgrt_train, serie_tgrt_test = serie_tgrt.split_before(dt_test_start)
if use_event_flag or events_to_consider or numeric_features:
    serie_cov_train, serie_cov_test = series_cov.split_before(dt_test_start)

In [ ]:
# Running the model
with mlflow.start_run() as mlflow_run:
    # Model parameters - already adjusted in optimization during model development
    lags = [-175, -168, -161, -154, -147, -140, -133, -126, -119, -112, -105, -98, -92, -91, -84, -77, -70, -63, -62, -56, -49, -42, -41, -37, -35, -34, -29, -28, -27, -26, -23, -22, -21, -20, -19, -16, -15, -14, -13, -12, -9, -8, -7, -6, -5, -2, -1]
    lags_past_covariates = lags
    lags_future_covariates = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    output_chunk_length = 1
    random_seed = 42
    n_estimators = 5000
    multi_models = False # If true, creates separate models to predict each future step. These models are encapsulated in the single final model.

    use_covariates = use_event_flag or events_to_consider or numeric_features
   
    # Model Creation and Training
    if use_covariates:
        quantiles = [0.05, 0.5, 0.95] # For a 90% confidence interval
        model = LightGBMModel(
                                lags=lags,
                                output_chunk_length=output_chunk_length,
                                lags_past_covariates=lags_past_covariates,
                                lags_future_covariates=lags_future_covariates,
                                random_state=random_seed,
                                n_estimators=n_estimators,
                                multi_models=multi_models,
                                force_row_wise=False,
                                force_col_wise=False,
                                #boosting_type='dart',
                                #num_leaves=40,
                                #max_depth=0,
                                learning_rate=None,
                                likelihood='quantile', 
                                quantiles=quantiles)
        model.fit(series=serie_tgrt_train, past_covariates=series_cov, future_covariates=series_cov[future_variables])
    else:
        model = create_model(use_covariates, lags, output_chunk_length, random_seed, n_estimators, multi_models)
        model.fit(series=serie_tgrt)
        
    # If using positive lags for future covariates
    if len(lags_future_covariates) > 1:
        serie_tgrt_test_cut = serie_tgrt_test[:-max(lags_future_covariates)]

    # Prediction and Validation
    if scale_trgt:
        df_preds, metrics_lists = predict_and_evaluate(model, serie_tgrt, serie_tgrt_test_cut, series_cov, scale_trgt, transformer_trgt, target_col)
    else:
        df_preds, mape_list, smape_list, ope_list, r2_list, rmse_list, wape_list, mape_list_acc, smape_list_acc,\
        ope_list_acc, r2_list_acc, rmse_list_acc, wape_list_acc = predict_and_evaluate(model, serie_tgrt, serie_tgrt_test_cut, series_cov, scale_trgt, None, target_col)

    # Adding actual values to dataframe with predictions
    if scale_trgt:
        serie_tgrt_test = transformer_trgt.inverse_transform(serie_tgrt_test)
    df_preds['NET_VALUE_real'] = serie_tgrt_test.pd_dataframe()[[target_col]]
    df_preds = df_preds.reset_index(drop=False).rename(columns={'index': 'SYSTEM_TIMESTAMP'})

    # Creating metrics dataframe per predicted day, disregarding weekends
    df_metrics = pd.DataFrame({'date': df_preds[df_preds.SYSTEM_TIMESTAMP.dt.weekday < 5].SYSTEM_TIMESTAMP,
                                'mape': metrics_lists['mape'],
                                'smape': metrics_lists['smape'],
                                'ope': metrics_lists['ope'],
                                'r2': metrics_lists['r2'],
                                'rmse': metrics_lists['rmse'],
                                'wape': metrics_lists['wape']})
    df_metrics['date'] = pd.to_datetime(df_metrics['date'])

    # Logging model parameters in MLflow
    mlflow.log_params({
        "n_test": n_test,
        "lags": lags,
        "output_chunk_length": output_chunk_length,
        "random_seed": random_seed,
        "n_estimators": n_estimators,
        "multi_models": multi_models,
        "dt_test_start": serie_tgrt_test[0].time_index.min(),
        "dt_test_end": serie_tgrt_test[0].time_index.max(),
        "scale_trgt": scale_trgt,
        "scale_method": scale_method,
        "dummy_day_of_week": dummy_day_of_week,
        "model_name": model_name
    })
    if use_covariates:
        mlflow.log_params({
            "lags_past_covariates": lags_past_covariates,
            "lags_future_covariates": lags_future_covariates,
            "scale_cov": scale_cov,
            "use_event_flag": use_event_flag,
            "numeric_features": numeric_features,
            "events_to_consider": events_to_consider
        })

    # Logging metrics mean and median
    mlflow.log_metrics({
        "mape_median": np.median(metrics_lists['mape']),
        "smape_median": np.median(metrics_lists['smape']),
        "ope_median": np.median(metrics_lists['ope']),
        "r2_median": np.median(metrics_lists['r2']),
        "rmse_median": np.median(metrics_lists['rmse']),
        "wape_median": np.median(metrics_lists['wape']),
        "mape_avg": np.average(metrics_lists['mape']),
        "smape_avg": np.average(metrics_lists['smape']),
        "ope_avg": np.average(metrics_lists['ope']),
        "r2_avg": np.average(metrics_lists['r2']),
        "rmse_avg": np.average(metrics_lists['rmse']),
        "wape_avg": np.average(metrics_lists['wape'])
    })

    # Logging prediction and metrics results
    df_preds.to_csv(artifacts_root + f'predictions_{name}.csv', index=False)
    mlflow.log_artifact(artifacts_root + f"predictions_{name}.csv")
    df_metrics.to_csv(artifacts_root + f'metrics_{name}.csv', index=False)
    mlflow.log_artifact(artifacts_root + f"metrics_{name}.csv")
    
    # Plot - Metrics Time Series for MAPE
    fig, ax = plt.subplots(figsize=(25, 10))
    df_metrics[['date', 'wape']].set_index('date').plot(ax=ax)
    plt.title('WAPE Time Series', fontsize=20, loc='center', weight='bold')
    ax.xaxis.set_major_locator(plt.MaxNLocator(15))
    ax.tick_params(axis='x', rotation=45)
    plt.savefig(artifacts_root + f'ts_metrics_{name}.png', bbox_inches='tight')
    mlflow.log_artifact(artifacts_root + f"ts_metrics_{name}.png")

    # Creating WAPE chart
    plt.figure(figsize=(15, 10))
    plt.title(f'WAPE Histogram - {name} - median:{round(np.median(metrics_lists["wape"]), 2)}')
    sns.histplot(remove_outliers_zscore(metrics_lists['wape']), kde = False)
    plt.axvline(x = np.median(remove_outliers_zscore(metrics_lists['wape'])), linestyle = '--')
    plt.xlabel("WAPE")
    plt.savefig(artifacts_root + f"hist_wape_{name}.png")
    mlflow.log_artifact(artifacts_root + f"hist_wape_{name}.png") 

    # Creating OPE chart
    plt.figure(figsize=(15, 10))
    plt.title(f'OPE Histogram - {name} - median:{round(np.median(metrics_lists["ope"]), 2)}')
    sns.histplot(remove_outliers_zscore(metrics_lists['ope']), kde = False)
    plt.axvline(x = np.median(remove_outliers_zscore(metrics_lists['ope'])), linestyle = '--')
    plt.xlabel("OPE")
    plt.savefig(artifacts_root + f"hist_ope_{name}.png")
    mlflow.log_artifact(artifacts_root + f"hist_ope_{name}.png") 

    # Calculating and logging prediction and actual decomposition
    decomposition_real = seasonal_decompose(df_preds[['NET_VALUE_real']], 
                                        model = 'additive', 
                                        period = 30,
                                        two_sided = False)

    decomposition_predicted = seasonal_decompose(df_preds[['NET_VALUE_predicted']], 
                                                model = 'additive', 
                                                period = 30,
                                                two_sided = False)

    df_components = pd.DataFrame(decomposition_real.trend)
    df_components.columns = ['trend_hist']
    df_components['seasonal_hist'] = decomposition_real.seasonal
    df_components['observed_hist'] = decomposition_real.observed
    df_components['resid_hist'] = decomposition_real.resid
    df_components['trend_pred'] = decomposition_predicted.trend
    df_components['seasonal_pred'] = decomposition_predicted.seasonal
    df_components['observed_pred'] = decomposition_predicted.observed
    df_components['resid_pred'] = decomposition_predicted.resid
    df_components.index = df_preds['SYSTEM_TIMESTAMP']

    fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(20, 12), sharex = True)
    fig.suptitle(f'Sellout and predicted components - {name}')
    df_components = df_components[-8*7:]
    df_components[['observed_hist', 'observed_pred']].plot(ax=axes[0], title = 'Observed')
    df_components[['trend_hist', 'trend_pred']].plot(ax=axes[1], title = 'Trend')
    df_components[['seasonal_hist', 'seasonal_pred']].plot(ax=axes[2], title = 'Seasonal')
    df_components[['resid_hist', 'resid_pred']].plot(ax=axes[3], title = 'Residual')
    fig.savefig(artifacts_root + f"components_real_predict_{name}.png")
    mlflow.log_artifact(artifacts_root + f"components_real_predict_{name}.png") 

    # Calculating and logging residuals
    df_preds['residuals'] = df_preds.NET_VALUE_predicted - df_preds.NET_VALUE_real
    plt.figure(figsize=(15, 10))
    plt.title(f'Residuals Histogram - {name} - median:{round(df_preds.residuals.median(), 2)}')
    sns.histplot(df_preds, x = 'residuals', kde = False)
    plt.axvline(x = df_preds.residuals.mean(), linestyle = '--')
    plt.xlabel("Residuals")
    plt.savefig(artifacts_root + f"hist_residuals_{name}.png")
    mlflow.log_artifact(artifacts_root + f"hist_residuals_{name}.png") 

    # Creating chart with residual autocorrelation
    plt.rc("figure",figsize=(20,7))
    acf = plot_acf(df_preds.residuals, title=f'Residuals Autocorrelation - {name}');
    acf.savefig(artifacts_root + f"ACF_residuals_agg_{name}.png")
    mlflow.log_artifact(artifacts_root + f"ACF_residuals_agg_{name}.png")

    # Verifying and logging residual autocorrelation
    # If pvalue is greater than 0.05, H0 is not rejected. H0: the model does not exhibit lack of fit
    # H0 suggests the model captured the autocorrelation structure present in data
    # Including pvalue_ljungbox_agg in metrics and logging
    resids = df_preds.residuals.values
    pvalue_ljungbox = round(acorr_ljungbox(x = resids).lb_pvalue.max(), 4) 
    mlflow.log_metric("pvalue_ljungbox", pvalue_ljungbox)


    # Performing training with all available data
    if use_covariates:
        quantiles = [0.05, 0.5, 0.95] # For a 90% confidence interval
        model = LightGBMModel(
                                lags=lags,
                                output_chunk_length=output_chunk_length,
                                lags_past_covariates=lags_past_covariates,
                                lags_future_covariates=lags_future_covariates,
                                random_state=random_seed,
                                n_estimators=n_estimators,
                                multi_models=multi_models,
                                force_row_wise=False,
                                force_col_wise=False,
                                #boosting_type='dart',
                                #num_leaves=40,
                                #max_depth=0,
                                learning_rate=None,
                                likelihood='quantile', 
                                quantiles=quantiles)
        model.fit(series=serie_tgrt, past_covariates=series_cov, future_covariates=series_cov[future_variables])
    else:
        model = create_model(use_covariates, lags, output_chunk_length, random_seed, n_estimators, multi_models)
        model.fit(series=serie_tgrt)

    # Logging final model
    mlflow.sklearn.log_model(model, model_name+'_'+name)
    
    # Registering model in Model Registry
    run_id = mlflow_run.info.run_id
    model_uri = f"runs:/{run_id}/{registered_model_name}"
    registered_model = mlflow.register_model(model_uri, registered_model_name)

    mlflow.end_run()